In [12]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Read the CSV files
triangles_df = pd.read_csv('triangles.csv', header=0, names=['v0', 'v1', 'v2'])
vertices_ecef_df = pd.read_csv('vertices_ecef.csv', header=0, names=['x', 'y', 'z'])
vertices_latlon_df = pd.read_csv('vertices_latlon.csv', header=0, names=['lat', 'lon', 'height'])


print(triangles_df.head())
print(vertices_ecef_df.head())
print(vertices_latlon_df.head())

# Extract the vertex indices for each triangle
i = triangles_df['v0'].values
j = triangles_df['v1'].values
k = triangles_df['v2'].values

# Extract the latitude, longitude, and height values
lat = vertices_latlon_df['lat'].values
lon = vertices_latlon_df['lon'].values
height = vertices_latlon_df['height'].values


x = vertices_ecef_df['x'].values
y = vertices_ecef_df['y'].values
z = vertices_ecef_df['z'].values



# Define the WGS84 ellipsoid parameters
a = 6378137.0  # semi-major axis (meters)
b = 6356752.314245  # semi-minor axis (meters)
e2 = (a**2 - b*2) / a**2  # Square of eccentricity

def find_intersection(x_p, y_p, z_p, normal_vector, semi_major, semi_minor):
    a = semi_major
    b = semi_minor

    # Coefficients of the quadratic equation
    A = (normal_vector[0]**2 / a**2) + (normal_vector[1]**2 / a**2) + (normal_vector[2]**2 / b**2)
    B = 2 * ((x_p * normal_vector[0] / a**2) + (y_p * normal_vector[1] / a**2) + (z_p * normal_vector[2] / b**2))
    C = (x_p**2 / a**2) + (y_p**2 / a**2) + (z_p**2 / b**2) - 1

    # Solve the quadratic equation for t
    t1 = (-B + np.sqrt(B**2 - 4 * A * C)) / (2 * A)
    t2 = (-B - np.sqrt(B**2 - 4 * A * C)) / (2 * A)

    # Choose the correct t (the one that gives the point on the ellipsoid surface)
    t = max(t1, t2)

    # Calculate the intersection point
    x_e = x_p + t * normal_vector[0]
    y_e = y_p + t * normal_vector[1]
    z_e = z_p + t * normal_vector[2]

    return x_e, y_e, z_e

# Convert lat, lon, height to ECEF coordinates
def latlon_to_ecef(lat, lon, height):
    # radius of curvature of the prime vertical section
    N = a**2 / np.hypot( a * np.cos(lat), b * np.sin(lat)
    )
    # Compute cartesian (geocentric) coordinates given (curvilinear) geodetic coordinates.
    x = (N + height) * np.cos(lat) * np.cos(lon)
    y = (N + height) * np.cos(lat) * np.sin(lon)
    z = (N * (b / a) ** 2 + height) * np.sin(lat)
    return x,y,z

x_, y_, z_ = latlon_to_ecef(lat, lon, height)

# Create the mesh plot
fig = go.Figure()

centroid_x = (np.max(x) + np.min(x)) /2;
centroid_y = (np.max(y) + np.min(y)) /2;
centroid_z =(np.max(z) + np.min(z)) /2;

centroid_x_ = (np.max(x_) + np.min(x_)) /2;
centroid_y_ = (np.max(y_) + np.min(y_)) /2;
centroid_z_ =(np.max(z_) + np.min(z_)) /2;


# Add the mesh with lighting
fig.add_trace(go.Mesh3d(
    x=x -  centroid_x,
    y=y -  centroid_y,
    z=z -  centroid_z,
    i=i,
    j=j,
    k=k,
    opacity=1,
    intensity=height,  # Use height values for shading
    colorscale='Viridis',  # Choose a colorscale
    colorbar=dict(title='Height'),
    lighting=dict(ambient=0.5, diffuse=0.7, specular=0.5, roughness=0.5, fresnel=0.5),
    showscale=True,
    showlegend=True,
))

# Create the ellipsoid surface
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 50)
u, v = np.meshgrid(u, v)
x_ellipsoid = a * np.cos(u) * np.sin(v)
y_ellipsoid = a * np.sin(u) * np.sin(v)
z_ellipsoid = b * np.cos(v)

# Add the ellipsoid surface
# fig.add_trace(go.Surface(
#     x=x_ellipsoid,
#     y=y_ellipsoid,
#     z=z_ellipsoid,
#     colorscale='Viridis',
#     opacity=0.1,
#     showscale=True,
#     showlegend=True,

#     contours = {
#         "x": {"show": True, "start": 0, "end": 1, "size": 1, "color":"red"},
#         "y": {"show": True, "start": 0, "end": 1, "size": 1, "color":"green"},
#         "z": {"show": True, "start": 0, "end": 1, "size": 1, "color":"blue"},
#     })),



print(centroid_x, centroid_y, centroid_z)
print(centroid_x_, centroid_y_, centroid_z_)

# Calculate the geodetic surface normal at the centroid
normal_vector = np.array([centroid_x / a**2, centroid_y / a**2, centroid_z / b**2])
normal_vector /= np.linalg.norm(normal_vector)

x_e, y_e, z_e = find_intersection(centroid_x_, centroid_y_, centroid_z_, normal_vector, a, b)


#Add a line from the origin to the centroid
fig.add_trace(go.Scatter3d(
    x=[x_e, centroid_x],
    y=[y_e, centroid_y],
    z=[z_e, centroid_z],
    mode='lines',
    line=dict(color='red', width=3),
    name='Origin to Centroid',
    showlegend=True
))

# Create contours every 30 degrees
contour_latitudes = np.arange(np.floor(lat[0])-1, np.floor(lat[0])+1, 0.5)  # Latitude from -90 to 90 degrees
contour_longitudes = np.arange(np.floor(lon[0])-1, np.floor(lon[0])+1, 0.5)  # Longitude from -180 to 180 degrees

# Create contour lines for latitude
contour_lines_latitude_x = np.outer(np.cos(np.radians(contour_latitudes)), np.cos(np.radians(contour_longitudes))) * a
contour_lines_latitude_y = np.outer(np.cos(np.radians(contour_latitudes)), np.sin(np.radians(contour_longitudes))) * a
contour_lines_latitude_z = np.outer(np.sin(np.radians(contour_latitudes)), np.ones(len(contour_longitudes))) * b

# Create contour lines for longitude
contour_lines_longitude_x = np.outer(np.cos(np.radians(contour_latitudes)), np.cos(np.radians(contour_longitudes))) * a
contour_lines_longitude_y = np.outer(np.cos(np.radians(contour_latitudes)), np.sin(np.radians(contour_longitudes))) * a
contour_lines_longitude_z = np.outer(np.sin(np.radians(contour_latitudes)), np.ones(len(contour_longitudes))) * b

# Create contour lines for latitude
contour_lines_latitude_x = []
contour_lines_latitude_y = []
contour_lines_latitude_z = []
for lat in contour_latitudes:
    lat_rad = np.radians(lat)
    x_lat = a * np.cos(lat_rad) * np.cos(np.radians(contour_longitudes))
    y_lat = a * np.cos(lat_rad) * np.sin(np.radians(contour_longitudes))
    z_lat = a * np.sin(lat_rad) * np.ones_like(contour_longitudes)
    contour_lines_latitude_x.extend(x_lat.tolist() + [None])
    contour_lines_latitude_y.extend(y_lat.tolist() + [None])
    contour_lines_latitude_z.extend(z_lat.tolist() + [None])

# Create contour lines for longitude
contour_lines_longitude_x = []
contour_lines_longitude_y = []
contour_lines_longitude_z = []
for lon in contour_longitudes:
    lon_rad = np.radians(lon)
    x_lon = a * np.cos(np.radians(contour_latitudes)) * np.cos(lon_rad)
    y_lon = a * np.cos(np.radians(contour_latitudes)) * np.sin(lon_rad)
    z_lon = a * np.sin(np.radians(contour_latitudes))
    contour_lines_longitude_x.extend(x_lon.tolist() + [None])
    contour_lines_longitude_y.extend(y_lon.tolist() + [None])
    contour_lines_longitude_z.extend(z_lon.tolist() + [None])


# Flatten the contour lines arrays
contour_lines_latitude_x = np.array(contour_lines_latitude_x).flatten()
contour_lines_latitude_y = np.array(contour_lines_latitude_y).flatten()
contour_lines_latitude_z = np.array(contour_lines_latitude_z).flatten()
contour_lines_longitude_x = np.array(contour_lines_longitude_x).flatten()
contour_lines_longitude_y = np.array(contour_lines_longitude_y).flatten()
contour_lines_longitude_z = np.array(contour_lines_longitude_z).flatten()

# Create contour traces
# fig.add_trace(go.Scatter3d(
#     x=contour_lines_latitude_x,
#     y=contour_lines_latitude_y,
#     z=contour_lines_latitude_z,
#     mode='lines',
#     line=dict(color='black', width=1),
#     showlegend=True
# ))

# fig.add_trace(go.Scatter3d(
#     x=contour_lines_longitude_x,
#     y=contour_lines_longitude_y,
#     z=contour_lines_longitude_z,
#     mode='lines',
#     line=dict(color='black', width=1),
#     showlegend=True
# ))


# Set up the layout for the 3D plot
fig.update_layout(
    title='3D Mesh Plot from Lat/Lon/Height Coordinates with Ellipsoid',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data',
         camera=dict(
           eye=dict(x=normal_vector[0], y=normal_vector[1], z=normal_vector[2]),  # Geodetic surface normal
           #center=dict(x=centroid_x, y=centroid_y, z=centroid_z),  # Center the camera on the mesh
           up=dict(x=normal_vector[0], y=normal_vector[1], z=normal_vector[2])  # Geodetic surface normal
)))

# Show the plot
fig.show()
print(fig.layout.scene)

   v0  v1  v2
0   0   1   2
1   0   2   3
2   2   1   4
3   5   2   6
4   2   5   3
              x             y             z
0 -4.147895e+06  2.866513e+06 -3.893927e+06
1 -4.147895e+06  2.866514e+06 -3.893926e+06
2 -4.147896e+06  2.866513e+06 -3.893926e+06
3 -4.147896e+06  2.866512e+06 -3.893927e+06
4 -4.147895e+06  2.866514e+06 -3.893925e+06
       lat       lon      height
0 -0.66087  2.536881  481.834824
1 -0.66087  2.536881  481.836042
2 -0.66087  2.536881  481.819595
3 -0.66087  2.536881  481.806194
4 -0.66087  2.536881  481.191556
-4147927.523411566 2866497.6250534467 -3893890.8841522606
-4147927.523410375 2866497.6250548214 -3893890.8841524064


layout.Scene({
    'aspectmode': 'data',
    'camera': {'eye': {'x': np.float64(-0.6494663421319739),
                       'y': np.float64(0.4488250377485516),
                       'z': np.float64(-0.613799279836434)},
               'up': {'x': np.float64(-0.6494663421319739),
                      'y': np.float64(0.4488250377485516),
                      'z': np.float64(-0.613799279836434)}},
    'xaxis': {'title': {'text': 'X (m)'}},
    'yaxis': {'title': {'text': 'Y (m)'}},
    'zaxis': {'title': {'text': 'Z (m)'}}
})
